We want to solve a 2-d problem with Gaussian increments over time and Laplacian increments over trials, with loss function (negative log-likelihood):
\begin{equation}
L(x,z;\Delta N) = \sum\limits_{k=1}^{K} \sum\limits_{r=1}^{R} \left[ -\Delta N_{k,r} \, (x_k + z_r) + \log(1+e^{x_k + z_r}) \right] + \frac{1}{2\sigma} \sum\limits_{k=2}^{K} (x_k - x_{k-1})^2 + \lambda \sum\limits_{r=2}^{R} |z_r - z_{r-1}|
\end{equation}

In order to improve convergence and allow the use of
Theorem 4.3 (Stationarity With Block Alternation)
from [Jacobson&Fessler, 2007] (also on our slack channel),
we replace the L1 norm with a Huber loss with small quadratic region (e.g., $c=10^{-3}$):
\begin{equation}
|x| \simeq
H_c(x) =
\begin{cases}
\frac12 (c + \frac{1}{c} x^2) & \text{if $|x| < c$} \\
x & \text{if $|x| \geq c$}.
\end{cases}
\end{equation}

We obtain:
\begin{equation}
L_c(x,z;\Delta N) = \sum\limits_{k=1}^{K} \sum\limits_{r=1}^{R} \left[ -\Delta N_{k,r} \, (x_k + z_r) + \log(1+e^{x_k + z_r}) \right] + \frac{1}{2\sigma} \sum\limits_{k=2}^{K} (x_k - x_{k-1})^2 + \lambda \sum\limits_{r=2}^{R} H_c(z_r - z_{r-1})
\end{equation}

We can now write a quadratic majorizer w.r.t. the variables $z$ as:
\begin{align}
\phi_z(z;\bar x, \bar z) &= \sum\limits_{k=1}^{K} \sum\limits_{r=1}^{R} \left[ -\Delta N_{k,r} \, (\bar x_k + z_r) + \log(1+e^{\bar x_k + \bar z_r}) + (1 + e^{-(\bar x_k + \bar z_r)})^{-1} (z_r - \bar z_r) + \frac{1}{8} (z_r - \bar z_r)^2 \right] \\
&+ \frac{1}{2\sigma} \sum\limits_{k=2}^{K} (\bar x_k - \bar x_{k-1})^2 + \lambda \sum\limits_{r=2}^{R} \frac12 \left[ \max(c, |\bar z_r - \bar z_{r-1}|) + \frac{(z_r - z_{r-1})^2}{\max(c, |\bar z_r - \bar z_{r-1}|)} \right] \\
&= \sum\limits_{k=1}^{K} \sum\limits_{r=1}^{R} \left[ -\Delta N_{k,r} \, z_r + \frac{z_r - \bar z_r}{1 + e^{-(\bar x_k + \bar z_r)}} + \frac{(z_r - \bar z_r)^2}{8} \right] + \lambda \sum\limits_{r=2}^{R} \frac{(z_r - z_{r-1})^2}{2 \max(c, |\bar z_r - \bar z_{r-1}|)} + \mathrm{const}
\end{align}
which is quadratic in $z$ and can be solved exactly in one iteration using a simple Kalman filter-smoother.
For the quadratic term of the likelihood term, we have used the following fact about the quadrature of the logistic loss:
\begin{equation}
\frac{d}{dx}(\log(1 + e^x)) = \frac{1}{e^x + 2 + e^{-x}} \leq 4.
\end{equation}
A less conservative, expansion-point-dependent quadratic approximation may be sought if needed.

A quadratic majorizer w.r.t. the variables $x$ is:
\begin{align}
\phi_x(x;\bar x, \bar z)
&= \sum\limits_{k=1}^{K} \sum\limits_{r=1}^{R} \left[ -\Delta N_{k,r} \, x_k + \frac{x_k - \bar x_k}{1 + e^{-(\bar x_k + \bar z_r)}} + \frac{(x_k - \bar x_k)^2}{8} \right] + \frac{1}{2\sigma} \sum\limits_{k=2}^{K} (x_k - x_{k-1})^2 + \mathrm{const}.
\end{align}

Because the function $L_c$ is coercive and the majorizers have uniform upper and lower curvature bounds, conditions (R1), (R2), (R3), (C4), (C5) and (C6) from [Jacobson&Fessler, 2007] hold. This guarantees the convergence of an algorithm that cyclically alternates MM steps over $x$ and $z$.

A possible advantage of this algorithm is that it uses one Kalman filter-smoother over $x$, then one over $z$ alternatively. In a sense, $x$ and $z$ are improved gradually together. Alternative solutions perform multiple Kalman iterations over $x$ for a fixed and possibly suboptimal value of $z$, therefore spending unnecessary effort trying to fine tune $x$ to refine a solution which is off anyway because of a subop$z$ (and vice versa).